# Normalize and build location

In [1]:
import pandas as pd
import re

DATASET_1_FILE = 'company_dataset_1.csv'
DATASET_2_FILE = 'company_dataset_2.csv'

df1 = pd.read_csv(DATASET_1_FILE)
df2 = pd.read_csv(DATASET_2_FILE)

def normalize_name(name):
    if pd.isna(name):
        return None
    name = name.lower()
    name = re.sub(r'[^a-z0-9 ]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

def normalize_text(x):
    if pd.isna(x):
        return ''
    x = str(x).lower()
    x = re.sub(r'\s+', ' ', x)
    x = x.strip()
    return x

def normalize_postcode(x):
    if pd.isna(x):
        return ''
    return re.sub(r'\s+', '', str(x).upper())

COUNTRY_MAP = {
    'canada': 'CA', 'ca': 'CA', 'CANADA': 'CA',
    'usa': 'US', 'us': 'US', 'united states': 'US'
}

STATE_MAP = {
    'ontario': 'ON', 'on': 'ON',
    'quebec': 'QC', 'qc': 'QC', 'québec': 'QC',
    'british columbia': 'BC', 'bc': 'BC',
    'TEXAS': 'TX', 'tx': 'TX'
}

def normalize_country(x):
    if pd.isna(x):
        return ''
    x_clean = str(x).strip().lower()
    return COUNTRY_MAP.get(x_clean, x_clean.upper())

def normalize_state(x):
    if pd.isna(x):
        return ''
    x_clean = str(x).strip().lower()
    return STATE_MAP.get(x_clean, x_clean.upper())

df1['company_norm'] = df1['custname'].apply(normalize_name)
df1['sCountry_norm'] = df1['sCountry'].apply(normalize_country)
df1['sProvState_norm'] = df1['sProvState'].apply(normalize_state)
df1['sCity_norm'] = df1['sCity'].apply(normalize_text)

df2['company_norm'] = df2['custname'].apply(normalize_name)
df2['country_norm'] = df2['country'].apply(normalize_country)
df2['state_norm'] = df2['state'].apply(normalize_state)
df2['city_norm'] = df2['city'].apply(normalize_text)

def build_location_tuple(df, zip_col, city_col, state_col, country_col):
    return list(
        zip(
            df[zip_col].apply(normalize_postcode),
            df[city_col],
            df[state_col],
            df[country_col],
        )
    )

df1['loc_tuple'] = build_location_tuple(df1, 'sPostalZip', 'sCity_norm', 'sProvState_norm', 'sCountry_norm')
df2['loc_tuple'] = build_location_tuple(df2, 'zip', 'city_norm', 'state_norm', 'country_norm')

# Merge dataset

In [2]:
df1_agg = (
    df1.groupby('company_norm', as_index=False)
       .agg(
           company_name_ds1=('company_norm', 'first'),
           locations_ds1=('loc_tuple', lambda x: list(set(x)))
       )
)

df2_agg = (
    df2.groupby('company_norm', as_index=False)
       .agg(
           company_name_ds2=('company_norm', 'first'),
           locations_ds2=('loc_tuple', lambda x: list(set(x)))
       )
)

merged = df1_agg.merge(df2_agg, on='company_norm', how='left')

# Building final dataset 

In [3]:
def get_overlapping_location_tiers(locs1, locs2):
    if not isinstance(locs2, list):
        return ''

    s1, s2 = set(map(tuple, locs1)), set(map(tuple, locs2))  # ensure tuples for set ops

    # Define tiers as (name, indices to compare)
    tiers = [
        ("postcode+city+state+country", (0, 1, 2, 3)),
        ("postcode+city+state", (0, 1, 2)),
        ("postcode+city", (0, 1)),
        ("postcode", (0,)),
        ("city+state+country", (1, 2, 3)),
        ("city+state", (1, 2)),
        ("city", (1,)),
        ("state+country", (2, 3)),
        ("state", (2,)),
        ("country", (3,)),
    ]

    for tier_name, indices in tiers:
        # Create sets of the values at the specified indices
        extract = lambda loc: tuple(loc[i] for i in indices)
        overlap = {extract(loc) for loc in s1} & {extract(loc) for loc in s2}

        if overlap:
            # Collect matching locations from locs1
            result = [', '.join(filter(None, loc)) for loc in s1 if extract(loc) in overlap]
            return '; '.join(result)

    return ''



merged['overlapping_location'] = merged.apply(
    lambda r: get_overlapping_location_tiers(r['locations_ds1'], r['locations_ds2']),
    axis=1
)

final_df = merged[[
    'company_name_ds1',
    'company_name_ds2',
    'locations_ds1',
    'locations_ds2',
    'overlapping_location'
]]


display(final_df.head(5))

final_df.to_csv('final_df.csv', index=False)

,company_name_ds1,company_name_ds2,locations_ds1,locations_ds2,overlapping_location
0,110 cvpw lp,NaN,"[(L3T0A1, markham, ON, )]",NaN,
1,110140 coxommepa road taguomex inc,NaN,"[(K2C0P9, ottawa, ON, )]",NaN,
2,1106832 ontario ltd,NaN,"[(L6J1H9, oakville, ON, )]",NaN,
3,1111429 ontario inc,1111429 ontario inc,"[(M9M2G4, north york, ON, CA)]","[(M9M2G4, north york, ON, CA)]","M9M2G4, north york, ON, CA"
4,1132 international monumaa inc,1132 international monumaa inc,"[(L6H0C3, oakville, ON, )]","[(L6H0C3, oakville, ON, CA)]","L6H0C3, oakville, ON"


# Metrics

In [4]:
total_ds1 = merged.shape[0]

matched = merged['locations_ds2'].apply(lambda x: isinstance(x, list) and len(x) > 0)
num_matched = matched.sum()

num_unmatched = (~matched).sum()

num_one_to_many = merged['locations_ds2'].apply(lambda x: isinstance(x, list) and len(x) > 1).sum()

print("=== Summary Metrics ===")
print(f'Total companies in Dataset 1: {total_ds1}')
print(f'Companies matched in Dataset 2: {num_matched} {round(100*num_matched/total_ds1,2)}%')
print(f'Companies unmatched: {num_unmatched} {round(100*num_unmatched/total_ds1,2)}%')
print(f'One-to-many matches: {num_one_to_many} {round(100*num_one_to_many/total_ds1,2)}%')


num_overlapping = merged['overlapping_location'].str.strip().astype(bool).sum()
print(f"Number of overlapping locations: {num_overlapping}")

=== Summary Metrics ===
Total companies in Dataset 1: 1013
Companies matched in Dataset 2: 425 41.95%
Companies unmatched: 588 58.05%
One-to-many matches: 35 3.46%
Number of overlapping locations: 423
